In [ ]:
import sys
print("Hello from Colab kernel!", sys.version)


In [ ]:
import platform
print("Python:", platform.python_version())

!nvidia-smi


In [3]:
!pip install -q pandas numpy scikit-learn gradio requests openpyxl


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import gradio as gr
import requests


In [1]:
!git clone -b dev https://github.com/Abhishekchoure01/crop-risk-predictor.git
%cd crop-risk-predictor
!git pull origin dev
!pip install -q -r requirements.txt
!ls -la src/

fatal: destination path 'crop-risk-predictor' already exists and is not an empty directory.
/content/crop-risk-predictor
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 30 (delta 11), reused 28 (delta 9), pack-reused 0 (from 0)
Unpacking objects: 100% (30/30), 8.60 KiB | 1.23 MiB/s, done.
From https://github.com/Abhishekchoure01/crop-risk-predictor
 * branch            dev        -> FETCH_HEAD
   6bd0ea5..1cd320e  dev        -> origin/dev
Updating 6bd0ea5..1cd320e
Fast-forward
 notebooks/crop_risk_colab.ipynb | 172 ++++++++++++++++++++++++++++++++++++++++
 requirements.txt                |   6 ++
 src/__init__.py                 |  11 +++
 src/app.py                      | 132 ++++++++++++++++++++++++++++++
 src/data_gen.py                 |  47 +++++++++++
 src/model.py                    | 114 ++++++++++++++++++++++++++
 src/risk_logic.py               |  75 ++++++++++++++++++
 7 files

In [4]:
import sys
import os

# Add the cloned repo to Python path
sys.path.insert(0, '/content/crop-risk-predictor')

from src.data_gen import generate_synthetic_data
from src.model import CropRiskModel
from src.risk_logic import explain_risk
from src.app import create_gradio_app

print("✓ All imports successful!")

✓ All imports successful!


In [5]:
# 1) Synthetic data
rng = np.random.default_rng(42)

def generate_synthetic_data(n=50):
    rows = []
    for _ in range(n):
        rainfall_pct = 100 + rng.normal(0, 20)
        heatwave_days = 4 + rng.normal(0, 3)
        dry_days = 10 + rng.normal(0, 5)
        humidity = 70 + rng.normal(0, 8)
        loss = (100 - rainfall_pct) * 0.4 + heatwave_days * 5 + dry_days * 1.5 + max(0, humidity - 70) * 0.3
        rows.append([rainfall_pct, heatwave_days, dry_days, humidity, loss])
    df = pd.DataFrame(rows, columns=["rainfall_pct","heatwave_days","dry_days","humidity","loss_pct"])
    return df

df = generate_synthetic_data()
df.head()


,rainfall_pct,heatwave_days,dry_days,humidity,loss_pct
0,106.094342,0.880048,13.752256,77.524518,24.848241
1,60.979296,0.093461,10.639202,67.470059,32.034392
2,99.663977,1.440868,14.396990,76.222335,30.800936
3,101.320614,7.381724,12.337547,63.125660,54.886693
4,107.375016,1.123352,14.392252,69.600593,24.255132


In [6]:
# 2) Train model
features = ["rainfall_pct","heatwave_days","dry_days","humidity"]
X = df[features]
y = df["loss_pct"]

model = LinearRegression().fit(X, y)
r2 = r2_score(y, model.predict(X))
r2


0.9985378346965189

In [ ]:
# ============================================================================
# DISTRICTS, CROPS, and WEATHER DATA (UNCHANGED)
# ============================================================================
DISTRICTS = ["Pune","Nagpur","Mumbai"]
CROPS = ["Rice","Wheat","Cotton"]

DISTRICT_WEATHER = {
    "Pune":  {"rainfall_pct": 100, "heatwave_days": 4, "dry_days": 10, "humidity": 70},
    "Nagpur":{"rainfall_pct": 95,  "heatwave_days": 6, "dry_days": 12, "humidity": 68},
    "Mumbai":{"rainfall_pct": 120, "heatwave_days": 2, "dry_days": 5,  "humidity": 80},
}

# ============================================================================
# PREDICT_RISK FUNCTION (UNCHANGED)
# ============================================================================
def predict_risk(district, crop):
    w = DISTRICT_WEATHER[district]
    X_pred = np.array([[w["rainfall_pct"], w["heatwave_days"], w["dry_days"], w["humidity"]]])
    loss = float(model.predict(X_pred)[0])
    
    contrib = {
        "rainfall_pct": max(0, (100 - w["rainfall_pct"]) * 0.4),
        "heatwave_days": w["heatwave_days"] * 5,
        "dry_days": w["dry_days"] * 1.5,
        "humidity": max(0, (w["humidity"] - 70) * 0.3),
    }
    top2 = sorted(contrib.items(), key=lambda x: x[1], reverse=True)[:2]
    severity = "🔴 HIGH" if loss > 30 else "🟡 MEDIUM" if loss > 15 else "🟢 LOW"
    
    lines = [
        f"{district} - {crop} Risk Assessment",
        f"🎯 Predicted Loss: {loss:.0f}% {severity}",
        "🔴 Top Risks:",
    ]
    for name, val in top2:
        lines.append(f"• {name}: {val:.0f}%")
    lines.append("✅ Actions:")
    lines.append("1. Switch to drought-resistant variety")
    lines.append("2. Prepare for early harvest")
    lines.append(f"📊 Model Confidence: R² = {r2:.2f}")
    return "\n".join(lines)

# ============================================================================
# MULTILINGUAL TRANSLATIONS (en, mr, hi)
# ============================================================================
TRANSLATIONS = {
    'en': {
        'title': '🌾 Crop Risk Predictor',
        'subtitle': 'AI-Powered Assessment for Indian Farmers',
        'hero_desc': 'Protect your harvest with intelligent weather-based risk analysis',
        'config': 'Configuration',
        'language': 'Language',
        'region': 'Select Your Region & Crop',
        'district': 'District',
        'crop': 'Crop',
        'predict': '🔮 Predict Risk',
        'results': 'Risk Assessment Results',
        'select_inputs': '👈 Select district, crop, and language to begin',
        'footer': '🔬 Model Performance: R² = {:.2f} | 🌐 Languages: English • मराठी • हिन्दी | 📱 Empowering Indian Farmers'
    },
    'mr': {
        'title': '🌾 पीक जोखीम भविष्यवाणी',
        'subtitle': 'भारतीय शेतकऱ्यांसाठी AI-संचालित मूल्यांकन',
        'hero_desc': 'हवामान-आधारित जोखीम विश्लेषणासह आपली पिक संरक्षित करा',
        'config': 'कॉन्फिगरेशन',
        'language': 'भाषा',
        'region': 'आपला प्रदेश आणि पीक निवडा',
        'district': 'जिल्हा',
        'crop': 'पीक',
        'predict': '🔮 धोका भविष्यवाणी',
        'results': 'धोका मूल्यांकन परिणाम',
        'select_inputs': '👈 सुरू करण्यासाठी जिल्हा, पीक आणि भाषा निवडा',
        'footer': '🔬 मॉडेल कार्यक्षमता: R² = {:.2f} | 🌐 भाषा: English • मराठी • हिन्दी | 📱 भारतीय शेतकऱ्यांना सशक्त करणे'
    },
    'hi': {
        'title': '🌾 फसल जोखिम भविष्यवाणी',
        'subtitle': 'भारतीय किसानों के लिए AI-संचालित आकलन',
        'hero_desc': 'मौसम-आधारित जोखिम विश्लेषण के साथ अपनी फसल की रक्षा करें',
        'config': 'कॉन्फ़िगरेशन',
        'language': 'भाषा',
        'region': 'अपना क्षेत्र और फसल चुनें',
        'district': 'जिला',
        'crop': 'फसल',
        'predict': '🔮 जोखिम भविष्यवाणी',
        'results': 'जोखिम आकलन परिणाम',
        'select_inputs': '👈 शुरू करने के लिए जिला, फसल और भाषा चुनें',
        'footer': '🔬 मॉडल प्रदर्शन: R² = {:.2f} | 🌐 भाषाएं: English • मराठी • हिन्दी | 📱 भारतीय किसानों को सशक्त बनाना'
    }
}

LANG_CODES = {'English': 'en', 'मराठी': 'mr', 'हिन्दी': 'hi'}
LANG_NAMES = {'en': 'English', 'mr': 'मराठी', 'hi': 'हिन्दी'}

# ============================================================================
# ENHANCED GRADIO APP WITH GLASS MORPHISM & MULTILINGUAL SUPPORT
# ============================================================================
def create_enhanced_app():
    custom_theme = gr.themes.Soft(primary_hue="green", secondary_hue="blue")
    
    with gr.Blocks(theme=custom_theme, css="""
    body {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    .glass-card, .gradio-group {
        backdrop-filter: blur(10px);
        background: rgba(255, 255, 255, 0.15) !important;
        border: 1px solid rgba(255, 255, 255, 0.3) !important;
        border-radius: 15px;
        padding: 20px;
        box-shadow: 0 8px 32px rgba(31, 38, 135, 0.37);
    }
    .hero-section {
        text-align: center;
        padding: 40px 20px;
        background: linear-gradient(135deg, rgba(255,255,255,0.15) 0%, rgba(255,255,255,0.05) 100%);
        border-radius: 20px;
        border: 1px solid rgba(255,255,255,0.3);
        margin-bottom: 30px;
        box-shadow: 0 8px 32px rgba(31, 38, 135, 0.37);
    }
    .hero-title {
        font-size: 2.5em;
        font-weight: bold;
        margin-bottom: 10px;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
        color: transparent;
    }
    .hero-subtitle {
        font-size: 1.1em;
        color: rgba(255,255,255,0.9);
        margin-bottom: 15px;
        font-weight: 500;
    }
    .result-card {
        background: linear-gradient(135deg, rgba(102, 126, 234, 0.15) 0%, rgba(118, 75, 162, 0.15) 100%);
        border: 2px solid rgba(102, 126, 234, 0.4) !important;
        border-radius: 15px;
        padding: 25px;
        font-family: 'Courier New', monospace;
        backdrop-filter: blur(10px);
        box-shadow: 0 8px 32px rgba(31, 38, 135, 0.37);
    }
    .gradio-button {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        font-size: 1.1em;
        padding: 12px 30px;
        font-weight: bold;
        border-radius: 10px;
        border: none;
        color: white;
    }
    .gradio-button:hover {
        opacity: 0.9;
        transform: translateY(-2px);
        box-shadow: 0 12px 24px rgba(102, 126, 234, 0.4);
    }
    """) as demo:
        
        # Language state
        current_lang = gr.State("en")
        
        # ====== HERO SECTION ======
        with gr.Group(elem_classes="hero-section"):
            hero_title = gr.HTML("<div class='hero-title'>🌾 Crop Risk Predictor</div>")
            hero_subtitle = gr.HTML("<div class='hero-subtitle'>AI-Powered Assessment for Indian Farmers</div>")
            hero_desc = gr.HTML("<div style='color: rgba(255,255,255,0.85); font-size: 1em;'>Protect your harvest with intelligent weather-based risk analysis</div>")
        
        # ====== CONTROLS SECTION ======
        with gr.Group(elem_classes="glass-card"):
            lang_label = gr.Markdown("### ⚙️ Configuration")
            lang_dd = gr.Dropdown(
                choices=['English', 'मराठी', 'हिन्दी'],
                value="English",
                label="Language / भाषा / भाषा",
                scale=1
            )
        
        with gr.Group(elem_classes="glass-card"):
            region_label = gr.Markdown("### 📍 Select Your Region & Crop")
            with gr.Row():
                dist_dd = gr.Dropdown(
                    DISTRICTS,
                    label="District",
                    scale=1,
                    interactive=True
                )
                crop_dd = gr.Dropdown(
                    CROPS,
                    label="Crop",
                    scale=1,
                    interactive=True
                )
        
        # ====== PREDICTION BUTTON ======
        predict_btn = gr.Button(
            "🔮 Predict Risk",
            variant="primary",
            scale=2
        )
        
        # ====== RESULTS SECTION ======
        with gr.Group(elem_classes="result-card"):
            result_label = gr.Markdown("### 📊 Risk Assessment Results")
            result_output = gr.Markdown(
                "👈 Select district, crop, and language to begin",
                label="Results"
            )
        
        # ====== FOOTER ======
        footer_text = gr.Markdown(
            "---\n🔬 **Model Performance**: R² = {:.2f} | "
            "🌐 **Languages**: English • मराठी • हिन्दी | "
            "📱 **Empowering Indian Farmers**".format(r2)
        )
        
        # ====== TRANSLATION UPDATE FUNCTION ======
        def update_ui_language(lang_name):
            lang_code = LANG_CODES[lang_name]
            trans = TRANSLATIONS[lang_code]
            
            return (
                gr.update(value=f"<div class='hero-title'>{trans['title']}</div>"),
                gr.update(value=f"<div class='hero-subtitle'>{trans['subtitle']}</div>"),
                gr.update(value=f"<div style='color: rgba(255,255,255,0.85); font-size: 1em;'>{trans['hero_desc']}</div>"),
                gr.update(value=f"### ⚙️ {trans['config']}"),
                gr.update(label=trans['language']),
                gr.update(value=f"### 📍 {trans['region']}"),
                gr.update(label=trans['district']),
                gr.update(label=trans['crop']),
                gr.update(value=trans['predict']),
                gr.update(value=f"### 📊 {trans['results']}"),
                gr.update(value=f"---\n{trans['footer']}")
            )
        
        # ====== EVENT HANDLERS ======
        def predict_and_update(district, crop, lang_name):
            if not district or not crop:
                lang_code = LANG_CODES[lang_name]
                return TRANSLATIONS[lang_code]['select_inputs']
            return predict_risk(district, crop)
        
        lang_dd.change(
            fn=update_ui_language,
            inputs=lang_dd,
            outputs=[hero_title, hero_subtitle, hero_desc, lang_label, lang_dd, 
                     region_label, dist_dd, crop_dd, predict_btn, result_label, footer_text]
        )
        
        predict_btn.click(
            fn=predict_and_update,
            inputs=[dist_dd, crop_dd, lang_dd],
            outputs=result_output
        )
        
        return demo

# ====== LAUNCH APP ======
app = create_enhanced_app()
app.launch(share=True)

/tmp/ipython-input-4005/3551176864.py:100: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=custom_theme, css="""
/tmp/ipython-input-4005/3551176864.py:100: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=custom_theme, css="""


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://88f1a62a09aa6405a3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [ ]:

# ============================================================================
# PHASE 2: ENHANCED DETAILS + CHARTS + REAL-TIME WEATHER (WITH TRANSLATIONS)
# ============================================================================
import plotly.graph_objects as go
from datetime import datetime
import pytz

# ============================================================================
# EXPANDED TRANSLATIONS FOR RESULTS
# ============================================================================
TRANSLATIONS_EXTENDED = {
    'en': {
        'title': '🌾 Crop Risk Predictor',
        'subtitle': 'AI-Powered Assessment for Indian Farmers',
        'hero_desc': 'Protect your harvest with intelligent weather-based risk analysis',
        'config': 'Configuration',
        'language': 'Language',
        'region': 'Select Your Region & Crop',
        'district': 'District',
        'crop': 'Crop',
        'predict': '🔮 Predict Risk',
        'results': 'Risk Assessment Results',
        'select_inputs': '👈 Select district, crop, and language to begin',
        'footer': '🔬 Model Performance: R² = {:.2f} | 🌐 Languages: English • मराठी • हिन्दी | 📱 Empowering Indian Farmers',
        # Result translations
        'risk_assessment': 'Risk Assessment',
        'predicted_loss': 'Predicted Loss',
        'top_risks': 'Top Risks',
        'actions': 'Actions',
        'action1': 'Switch to drought-resistant variety',
        'action2': 'Prepare for early harvest',
        'confidence': 'Model Confidence',
        'critical': '🔴 CRITICAL',
        'high': '🔴 HIGH',
        'medium': '🟡 MEDIUM',
        'low': '🟢 LOW',
        # Detailed report translations
        'detailed_analysis': 'DETAILED RISK ANALYSIS',
        'risk_level': 'Risk Assessment',
        'confidence_level': 'Confidence Level',
        'analysis_time': 'Analysis Time',
        'top_risk_factors': 'Top Risk Factors',
        'impact': 'impact',
        'current_weather': 'Current Weather Conditions',
        'temperature': 'Temperature',
        'humidity': 'Humidity',
        'location': 'Location',
        'weather_parameters': 'Weather Parameters',
        'metric': 'Metric', 'value': 'Value', 'status': 'Status',
        'rainfall': 'Rainfall', 'normal': '✅ Normal', 'below_normal': '⚠️ Below Normal', 'critical_short': '🚨 Critical',
        'heatwave_days': 'Heatwave Days', 'low': '✅ Low', 'moderate': '⚠️ Moderate', 'high': '🚨 High',
        'dry_days': 'Dry Days',
        'humidity_label': 'Humidity', 'good': '✅ Good', 'humidity_low': '⚠️ Low',
        'of_normal': '% of normal',
        'days': 'days',
        'recommended_actions': 'Recommended Actions',
        'irrigation': 'Irrigation: Increase watering frequency by 20-30%',
        'crop_variety': 'Crop Variety: Switch to drought-resistant varieties',
        'harvest': 'Harvest: Consider early harvest if loss > 40%',
        'monitoring': 'Monitoring: Daily weather monitoring recommended',
        'insurance': 'Insurance: Check crop insurance eligibility',
        'model_info': 'Model Information',
        'algorithm': 'Algorithm: Linear Regression',
        'training_samples': 'Training Samples: 50',
        'features': 'Features: Rainfall, Heatwave Days, Dry Days, Humidity',
        'accuracy': 'Accuracy',
        'maharashtra': 'Maharashtra',
    },
    'mr': {
        'title': '🌾 पीक जोखीम भविष्यवाणी',
        'subtitle': 'भारतीय शेतकऱ्यांसाठी AI-संचालित मूल्यांकन',
        'hero_desc': 'हवामान-आधारित जोखीम विश्लेषणासह आपली पिक संरक्षित करा',
        'config': 'कॉन्फिगरेशन',
        'language': 'भाषा',
        'region': 'आपला प्रदेश आणि पीक निवडा',
        'district': 'जिल्हा',
        'crop': 'पीक',
        'predict': '🔮 धोका भविष्यवाणी',
        'results': 'धोका मूल्यांकन परिणाम',
        'select_inputs': '👈 सुरू करण्यासाठी जिल्हा, पीक आणि भाषा निवडा',
        'footer': '🔬 मॉडेल कार्यक्षमता: R² = {:.2f} | 🌐 भाषा: English • मराठी • हिन्दी | 📱 भारतीय शेतकऱ्यांना सशक्त करणे',
        # Result translations
        'risk_assessment': 'धोका मूल्यांकन',
        'predicted_loss': 'अंदाजित नुकसान',
        'top_risks': 'शीर्ष धोके',
        'actions': 'कृती',
        'action1': 'सूखा-प्रतिरोधी वाणिज्यात बदल करा',
        'action2': 'लवकर कापणीसाठी तयार रहा',
        'confidence': 'मॉडेल आत्मविश्वास',
        'critical': '🔴 गंभीर',
        'high': '🔴 उच्च',
        'medium': '🟡 मध्यम',
        'low': '🟢 कमी',
        # Detailed report translations
        'detailed_analysis': 'तपशील धोका विश्लेषण',
        'risk_level': 'धोका मूल्यांकन',
        'confidence_level': 'आत्मविश्वास स्तर',
        'analysis_time': 'विश्लेषण वेळ',
        'top_risk_factors': 'शीर्ष धोके घटक',
        'impact': 'प्रभाव',
        'current_weather': 'वर्तमान हवामान स्थिती',
        'temperature': 'तापमान',
        'humidity': 'आर्द्रता',
        'location': 'स्थान',
        'weather_parameters': 'हवामान मापदंड',
        'metric': 'मेट्रिक', 'value': 'मूल्य', 'status': 'स्थिति',
        'rainfall': 'पाऊस', 'normal': '✅ सामान्य', 'below_normal': '⚠️ सामान्यपेक्षा कमी', 'critical_short': '🚨 गंभीर',
        'heatwave_days': 'उष्ण लाटेचे दिवस', 'low': '✅ कमी', 'moderate': '⚠️ मध्यम', 'high': '🚨 उच्च',
        'dry_days': 'सूख्या दिवस',
        'humidity_label': 'आर्द्रता', 'good': '✅ चांगले', 'humidity_low': '⚠️ कमी',
        'of_normal': 'सामान्यचे %',
        'days': 'दिवस',
        'recommended_actions': 'शिफारस केलेल्या कृती',
        'irrigation': 'सिंचन: पाणी देण्याची वारंवारता २०-३० % वाढवा',
        'crop_variety': 'पीक वाणिज्य: सूखा-प्रतिरोधी वाणिज्यात बदल करा',
        'harvest': 'कापणी: नुकसान > ४०% असल्यास लवकर कापणी विचारात घ्या',
        'monitoring': 'निरीक्षण: दैनिक हवामान निरीक्षण शिफारस केली जाते',
        'insurance': 'विमा: पीक विमा पात्रता तपासा',
        'model_info': 'मॉडेल माहिती',
        'algorithm': 'अल्गोरिदम: रेखीय प्रतिगमन',
        'training_samples': 'प्रशिक्षण नमुने: ५०',
        'features': 'वैशिष्ट्ये: पाऊस, उष्ण लाट दिवस, सूख्या दिवस, आर्द्रता',
        'accuracy': 'अचूकता',
        'maharashtra': 'महाराष्ट्र',
    },
    'hi': {
        'title': '🌾 फसल जोखिम भविष्यवाणी',
        'subtitle': 'भारतीय किसानों के लिए AI-संचालित आकलन',
        'hero_desc': 'मौसम-आधारित जोखिम विश्लेषण के साथ अपनी फसल की रक्षा करें',
        'config': 'कॉन्फ़िगरेशन',
        'language': 'भाषा',
        'region': 'अपना क्षेत्र और फसल चुनें',
        'district': 'जिला',
        'crop': 'फसल',
        'predict': '🔮 जोखिम भविष्यवाणी',
        'results': 'जोखिम आकलन परिणाम',
        'select_inputs': '👈 शुरू करने के लिए जिला, फसल और भाषा चुनें',
        'footer': '🔬 मॉडल प्रदर्शन: R² = {:.2f} | 🌐 भाषाएं: English • मराठी • हिन्दी | 📱 भारतीय किसानों को सशक्त बनाना',
        # Result translations
        'risk_assessment': 'जोखिम मूल्यांकन',
        'predicted_loss': 'अनुमानित नुकसान',
        'top_risks': 'शीर्ष जोखिमें',
        'actions': 'कार्य',
        'action1': 'सूखा-प्रतिरोधी किस्म में बदलाव करें',
        'action2': 'जल्दी कटाई के लिए तैयार रहें',
        'confidence': 'मॉडेल विश्वास',
        'critical': '🔴 गंभीर',
        'high': '🔴 उच्च',
        'medium': '🟡 मध्यम',
        'low': '🟢 कम',
        # Detailed report translations
        'detailed_analysis': 'विस्तृत जोखिम विश्लेषण',
        'risk_level': 'जोखिम मूल्यांकन',
        'confidence_level': 'विश्वास स्तर',
        'analysis_time': 'विश्लेषण समय',
        'top_risk_factors': 'शीर्ष जोखिम कारक',
        'impact': 'प्रभाव',
        'current_weather': 'वर्तमान मौसम की स्थिति',
        'temperature': 'तापमान',
        'humidity': 'आर्द्रता',
        'location': 'स्थान',
        'weather_parameters': 'मौसम पैरामीटर',
        'metric': 'मीट्रिक', 'value': 'मान', 'status': 'स्थिति',
        'rainfall': 'वर्षा', 'normal': '✅ सामान्य', 'below_normal': '⚠️ सामान्य से नीचे', 'critical_short': '🚨 गंभीर',
        'heatwave_days': 'गर्मी लहर के दिन', 'low': '✅ कम', 'moderate': '⚠️ मध्यम', 'high': '🚨 उच्च',
        'dry_days': 'सूखे दिन',
        'humidity_label': 'आर्द्रता', 'good': '✅ अच्छा', 'humidity_low': '⚠️ कम',
        'of_normal': 'सामान्य का %',
        'days': 'दिन',
        'recommended_actions': 'अनुशंसित कार्य',
        'irrigation': 'सिंचाई: पानी देने की आवृत्ति में 20-30% वृद्धि करें',
        'crop_variety': 'फसल किस्म: सूखा-प्रतिरोधी किस्मों में बदलाव करें',
        'harvest': 'कटाई: यदि नुकसान > 40% है तो जल्दी कटाई करने पर विचार करें',
        'monitoring': 'निगरानी: दैनिक मौसम निगरानी की सिफारिश की जाती है',
        'insurance': 'बीमा: फसल बीमा पात्रता की जांच करें',
        'model_info': 'मॉडल जानकारी',
        'algorithm': 'एल्गोरिदम: रैखिक प्रतिगमन',
        'training_samples': 'प्रशिक्षण नमूने: 50',
        'features': 'विशेषताएं: वर्षा, गर्मी लहर दिन, सूखे दिन, आर्द्रता',
        'accuracy': 'सटीकता',
        'maharashtra': 'महाराष्ट्र',
    }
}

def get_current_weather(lat, lon):
    """Fetch current weather from Open-Meteo API (free, no API key needed)"""
    try:
        import requests
        url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current=temperature_2m,relative_humidity_2m"
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            data = response.json()
            current = data.get('current', {})
            return {
                'temp': current.get('temperature_2m', 'N/A'),
                'humidity': current.get('relative_humidity_2m', 'N/A')
            }
    except:
        pass
    return {'temp': 'N/A', 'humidity': 'N/A'}

# District coordinates (lat, lon)
DISTRICT_COORDS = {
    "Pune": (18.5204, 73.8567),
    "Nagpur": (21.1458, 79.0882),
    "Mumbai": (19.0760, 72.8777),
}

def create_risk_chart(district, crop):
    """Create interactive pie chart showing risk contribution breakdown"""
    w = DISTRICT_WEATHER[district]
    
    contrib = {
        "Rainfall Deficit": max(0, (100 - w["rainfall_pct"]) * 0.4),
        "Heatwave": w["heatwave_days"] * 5,
        "Dry Days": w["dry_days"] * 1.5,
        "Low Humidity": max(0, (w["humidity"] - 70) * 0.3),
    }
    
    # Filter out zero contributions
    contrib = {k: v for k, v in contrib.items() if v > 0}
    names = list(contrib.keys())
    values = list(contrib.values())
    colors = ["#FF6B6B", "#FFA500", "#FFD93D", "#6BCB77"]
    
    fig = go.Figure(data=[go.Pie(
        labels=names,
        values=values,
        marker=dict(colors=colors, line=dict(color="white", width=2)),
        textposition="inside",
        textinfo="percent+label",
        hovertemplate="<b>%{label}</b><br>Impact: %{value:.1f}%<extra></extra>",
    )])
    
    fig.update_layout(
        title=f"Risk Breakdown: {district} - {crop}",
        font=dict(size=12, family="Arial, sans-serif"),
        showlegend=True,
        height=400,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        font_color="white",
    )
    
    return fig

def predict_risk_translated(district, crop, lang_code='en'):
    """Generate risk prediction with full translations"""
    t = TRANSLATIONS_EXTENDED[lang_code]
    
    w = DISTRICT_WEATHER[district]
    X_pred = np.array([[w["rainfall_pct"], w["heatwave_days"], w["dry_days"], w["dry_days"], w["humidity"]]])
    loss = float(model.predict(X_pred)[0])
    
    contrib = {
        "rainfall_pct": max(0, (100 - w["rainfall_pct"]) * 0.4),
        "heatwave_days": w["heatwave_days"] * 5,
        "dry_days": w["dry_days"] * 1.5,
        "humidity": max(0, (w["humidity"] - 70) * 0.3),
    }
    top2 = sorted(contrib.items(), key=lambda x: x[1], reverse=True)[:2]
    
    if loss > 60:
        severity = t['critical']
    elif loss > 30:
        severity = t['high']
    elif loss > 15:
        severity = t['medium']
    else:
        severity = t['low']
    
    lines = [
        f"{district} - {crop} {t['risk_assessment']}",
        f"🎯 {t['predicted_loss']}: {loss:.0f}% {severity}",
        f"🔴 {t['top_risks']}:",
    ]
    for name, val in top2:
        lines.append(f"• {name}: {val:.0f}%")
    lines.append(f"✅ {t['actions']}:")
    lines.append(f"1. {t['action1']}")
    lines.append(f"2. {t['action2']}")
    lines.append(f"📊 {t['confidence']}: R² = {r2:.2f}")
    return "\n".join(lines)

def generate_detailed_report(district, crop, lang_code='en'):
    """Generate comprehensive report with charts and metrics - FULLY TRANSLATED"""
    t = TRANSLATIONS_EXTENDED[lang_code]
    w = DISTRICT_WEATHER[district]
    lat, lon = DISTRICT_COORDS[district]
    weather = get_current_weather(lat, lon)
    
    X_pred = np.array([[w["rainfall_pct"], w["heatwave_days"], w["dry_days"], w["humidity"]]])
    loss = float(model.predict(X_pred)[0])
    
    # Calculate risk factors
    contrib = {
        t['rainfall']: max(0, (100 - w["rainfall_pct"]) * 0.4),
        t['heatwave_days']: w["heatwave_days"] * 5,
        t['dry_days']: w["dry_days"] * 1.5,
        t['humidity_label']: max(0, (w["humidity"] - 70) * 0.3),
    }
    
    top_factors = sorted(contrib.items(), key=lambda x: x[1], reverse=True)[:2]
    
    if loss > 60:
        severity = t['critical']
    elif loss > 30:
        severity = t['high']
    elif loss > 15:
        severity = t['medium']
    else:
        severity = t['low']
    
    # IST timezone
    ist = pytz.timezone('Asia/Kolkata')
    timestamp = datetime.now(ist).strftime("%b %d, %Y %I:%M %p IST")
    
    # Rainfall status
    if w["rainfall_pct"] >= 80:
        rainfall_status = t['normal']
    elif w["rainfall_pct"] >= 50:
        rainfall_status = t['below_normal']
    else:
        rainfall_status = t['critical_short']
    
    # Heatwave status
    if w["heatwave_days"] < 5:
        heatwave_status = t['low']
    elif w["heatwave_days"] < 10:
        heatwave_status = t['moderate']
    else:
        heatwave_status = t['high']
    
    # Dry days status
    if w["dry_days"] < 5:
        dry_status = t['low']
    elif w["dry_days"] < 10:
        dry_status = t['moderate']
    else:
        dry_status = t['high']
    
    # Humidity status
    humidity_status = t['good'] if w["humidity"] >= 60 else t['humidity_low']
    
    # Build detailed report
    report = f"""
## 📊 {t['detailed_analysis']}
### {district} - {crop}

---

### 🎯 **{t['risk_level']}**
- **{t['predicted_loss']}**: {loss:.1f}% {severity}
- **{t['confidence_level']}**: R² = {r2:.2f} ± 0.05
- **{t['analysis_time']}**: {timestamp}

---

### 🔴 **{t['top_risk_factors']}**
"""
    
    for i, (factor, impact) in enumerate(top_factors, 1):
        report += f"**{i}. {factor}**: {impact:.0f}% {t['impact']}\n"
    
    report += f"""
---

### 🌡️ **{t['current_weather']}**
- **{t['temperature']}}**: {weather['temp']}°C
- **{t['humidity']}**: {weather['humidity']}%
- **{t['location']}}**: {district}, {t['maharashtra']}

---

### 📈 **{t['weather_parameters']}**
| {t['metric']} | {t['value']} | {t['status']} |
|--------|-------|--------|
| {t['rainfall']} | {w["rainfall_pct"]:.0f}{t['of_normal']} | {rainfall_status} |
| {t['heatwave_days']} | {w["heatwave_days"]:.0f} {t['days']} | {heatwave_status} |
| {t['dry_days']} | {w["dry_days"]:.0f} {t['days']} | {dry_status} |
| {t['humidity_label']} | {w["humidity"]:.0f}% | {humidity_status} |

---

### ✅ **{t['recommended_actions']}**
1. **{t['irrigation']}**
2. **{t['crop_variety']}**
3. **{t['harvest']}**
4. **{t['monitoring']}**
5. **{t['insurance']}**

---

### 📡 **{t['model_info']}**
- **{t['algorithm']}}**
- **{t['training_samples']}}**
- **{t['features']}}**
- **{t['accuracy']}}**: R² = {r2:.4f}
"""
    
    return report

def create_enhanced_app_phase2():
    """Enhanced app with detailed analysis, charts, and real-time weather - WITH FULL TRANSLATIONS"""
    custom_theme = gr.themes.Soft(primary_hue="green", secondary_hue="blue")
    
    with gr.Blocks(theme=custom_theme, css="""
    body {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    .glass-card, .gradio-group {
        backdrop-filter: blur(10px);
        background: rgba(255, 255, 255, 0.15) !important;
        border: 1px solid rgba(255, 255, 255, 0.3) !important;
        border-radius: 15px;
        padding: 20px;
        box-shadow: 0 8px 32px rgba(31, 38, 135, 0.37);
    }
    .hero-section {
        text-align: center;
        padding: 40px 20px;
        background: linear-gradient(135deg, rgba(255,255,255,0.15) 0%, rgba(255,255,255,0.05) 100%);
        border-radius: 20px;
        border: 1px solid rgba(255,255,255,0.3);
        margin-bottom: 30px;
        box-shadow: 0 8px 32px rgba(31, 38, 135, 0.37);
    }
    .hero-title {
        font-size: 2.5em;
        font-weight: bold;
        margin-bottom: 10px;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
        color: transparent;
    }
    .hero-subtitle {
        font-size: 1.1em;
        color: rgba(255,255,255,0.9);
        margin-bottom: 15px;
        font-weight: 500;
    }
    .result-card {
        background: linear-gradient(135deg, rgba(102, 126, 234, 0.15) 0%, rgba(118, 75, 162, 0.15) 100%);
        border: 2px solid rgba(102, 126, 234, 0.4) !important;
        border-radius: 15px;
        padding: 25px;
        backdrop-filter: blur(10px);
        box-shadow: 0 8px 32px rgba(31, 38, 135, 0.37);
    }
    .gradio-button {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        font-size: 1.1em;
        padding: 12px 30px;
        font-weight: bold;
        border-radius: 10px;
        border: none;
        color: white;
    }
    .gradio-button:hover {
        opacity: 0.9;
        transform: translateY(-2px);
        box-shadow: 0 12px 24px rgba(102, 126, 234, 0.4);
    }
    """) as demo:
        
        # ====== HERO SECTION ======
        with gr.Group(elem_classes="hero-section"):
            hero_title = gr.HTML("<div class='hero-title'>🌾 Crop Risk Predictor</div>")
            hero_subtitle = gr.HTML("<div class='hero-subtitle'>AI-Powered Assessment for Indian Farmers</div>")
            hero_desc = gr.HTML("<div style='color: rgba(255,255,255,0.85); font-size: 1em;'>Protect your harvest with intelligent weather-based risk analysis</div>")
        
        # ====== CONTROLS SECTION ======
        with gr.Group(elem_classes="glass-card"):
            lang_label = gr.Markdown("### ⚙️ Configuration")
            lang_dd = gr.Dropdown(
                choices=['English', 'मराठी', 'हिन्दी'],
                value="English",
                label="Language / भाषा / भाषा",
                scale=1
            )
        
        with gr.Group(elem_classes="glass-card"):
            region_label = gr.Markdown("### 📍 Select Your Region & Crop")
            with gr.Row():
                dist_dd = gr.Dropdown(
                    DISTRICTS,
                    label="District",
                    scale=1,
                    interactive=True
                )
                crop_dd = gr.Dropdown(
                    CROPS,
                    label="Crop",
                    scale=1,
                    interactive=True
                )
        
        # ====== PREDICTION BUTTON ======
        predict_btn = gr.Button(
            "🔮 Predict Risk",
            variant="primary",
            scale=2
        )
        
        # ====== QUICK RESULTS SECTION ======
        with gr.Group(elem_classes="result-card"):
            result_label = gr.Markdown("### 📊 Risk Assessment Results")
            result_output = gr.Markdown(
                "👈 Select district, crop, and language to begin",
                label="Results"
            )
        
        # ====== PHASE 2: DETAILED ANALYSIS ======
        with gr.Group(elem_classes="result-card"):
            gr.Markdown("### 📈 Detailed Analysis")
            
            with gr.Tabs():
                with gr.Tab("📊 Risk Chart"):
                    chart_output = gr.Plot(label="Risk Breakdown")
                
                with gr.Tab("📋 Full Report"):
                    report_output = gr.Markdown(
                        "👈 Generate prediction to see detailed report",
                        label="Detailed Report"
                    )
        
        # ====== FOOTER ======
        footer_text = gr.Markdown(
            "---\n🔬 **Model Performance**: R² = {:.2f} | "
            "🌐 **Languages**: English • मराठी • हिन्दी | "
            "📱 **Empowering Indian Farmers**".format(r2)
        )
        
        # ====== TRANSLATION UPDATE FUNCTION ======
        def update_ui_language(lang_name):
            lang_code = LANG_CODES[lang_name]
            trans = TRANSLATIONS_EXTENDED[lang_code]
            
            return (
                gr.update(value=f"<div class='hero-title'>{trans['title']}</div>"),
                gr.update(value=f"<div class='hero-subtitle'>{trans['subtitle']}</div>"),
                gr.update(value=f"<div style='color: rgba(255,255,255,0.85); font-size: 1em;'>{trans['hero_desc']}</div>"),
                gr.update(value=f"### ⚙️ {trans['config']}"),
                gr.update(label=trans['language']),
                gr.update(value=f"### 📍 {trans['region']}"),
                gr.update(label=trans['district']),
                gr.update(label=trans['crop']),
                gr.update(value=trans['predict']),
                gr.update(value=f"### 📊 {trans['results']}"),
                gr.update(value=f"---\n{trans['footer']}")
            )
        
        # ====== EVENT HANDLERS ======
        def predict_with_details(district, crop, lang_name):
            if not district or not crop:
                lang_code = LANG_CODES[lang_name]
                return (
                    TRANSLATIONS_EXTENDED[lang_code]['select_inputs'],
                    None,
                    f"👈 {TRANSLATIONS_EXTENDED[lang_code]['select_inputs']}"
                )
            
            lang_code = LANG_CODES[lang_name]
            
            # Quick result (TRANSLATED)
            quick_result = predict_risk_translated(district, crop, lang_code)
            
            # Detailed report (TRANSLATED)
            detailed = generate_detailed_report(district, crop, lang_code)
            
            # Risk chart
            chart = create_risk_chart(district, crop)
            
            return quick_result, chart, detailed
        
        lang_dd.change(
            fn=update_ui_language,
            inputs=lang_dd,
            outputs=[hero_title, hero_subtitle, hero_desc, lang_label, lang_dd, 
                     region_label, dist_dd, crop_dd, predict_btn, result_label, footer_text]
        )
        
        predict_btn.click(
            fn=predict_with_details,
            inputs=[dist_dd, crop_dd, lang_dd],
            outputs=[result_output, chart_output, report_output]
        )
        
        return demo

# ====== LAUNCH ENHANCED APP WITH PHASE 2 ======
print("🚀 Launching Phase 2 app with full translations...")
app_phase2 = create_enhanced_app_phase2()
app_phase2.launch(share=True)

🚀 Launching Phase 2 app with charts and detailed analysis...


/tmp/ipython-input-4005/2002991738.py:156: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=custom_theme, css="""
/tmp/ipython-input-4005/2002991738.py:156: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(theme=custom_theme, css="""


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://84f4d5aa35d4b143a2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

